# Raw Data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

path = 'Data/DAC_NationalDownloadableFile.csv'
RawData = pd.read_csv(path, low_memory = False, encoding = 'ISO-8859-1')
RawData.columns = [i.strip() for i in RawData.columns]
RawData

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,Grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,ind_assgn,grp_assgn,adrs_id
0,1225027097,42308652,I20071115000304,WELCH,CHARLES,G,NaN,M,NaN,CREIGHTON UNIVERSITY SCHOOL OF MEDICINE,1991.0,GENERAL SURGERY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,707 SHERIDAN AVE,NaN,NaN,CODY,WY,824143409,NaN,531312.0,CODY REGIONAL HEALTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,WY824143409CO707XXAVEX300
1,1225065105,840273538,I20040614001356,STEINBERGER,DEAN,S,NaN,M,OD,PENNSYLVANIA COLLEGE OF OPTOMETRY,1997.0,OPTOMETRY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1014 S WESTLAKE BLVD,SUITE 10,NaN,WESTLAKE VILLAGE,CA,913613131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,CA913613131WE1014XBLVD401
2,1225064058,8022147263,I20110601000900,VOIGHT,NANCY,L,NaN,F,NaN,MICHIGAN STATE UNIVERSITY COLLEGE OF HUMAN MED...,1975.0,"PSYCHOLOGIST, CLINICAL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,570 MEMORIAL CIR,SUITE 150,NaN,ORMOND BEACH,FL,321745062,3.866729e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,FL321745062OR570XXCIRX302
3,1225022437,2961495601,I20040419000713,GORCZYNSKI,AMY,V,NaN,F,MD,OTHER,1993.0,INTERNAL MEDICINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 ELLINWOOD CT,NaN,NaN,NEW HARTFORD,NY,134131108,NaN,330044.0,FAXTON-ST LUKE'S HEALTHCARE,330245,ST ELIZABETH MEDICAL CENTER,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,NY134131108NE1XXXXCTXX300
4,1225039282,1557559507,I20101217000567,GOLZ,MICHAEL,W,NaN,M,NaN,OTHER,1983.0,CHIROPRACTIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,719 FRANKLIN AVE,NaN,NaN,FRANKLIN LAKES,NJ,074171920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,M,NJ074171920FR719XXAVEX300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286116,1891957023,840366449,I20110216000407,ATHERTON,STACY,M,NaN,F,NaN,TEMPLE UNIVERSITY SCHOOL OF MEDICINE,2008.0,PODIATRY,NaN,NaN,NaN,NaN,NaN,FOOT AND ANKLE CENTER OF THE ROCKIES LLC,9.931538e+09,6.0,4600 HALE PKWY,SUITE 440,NaN,DENVER,CO,802204000,3.033214e+09,60030.0,MCKEE MEDICAL CENTER,060003,LONGMONT UNITED HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,CO802204000DE4600XPKWY311
2286117,1730110693,2466488291,I20050713000678,MASTIN,TERRYE,T,NaN,F,NP,"UNIVERSITY OF TENNESSEE, HSC, COLLEGE OF MEDICINE",1987.0,NURSE PRACTITIONER,NaN,NaN,NaN,NaN,NaN,BLUFF CITY MULTISPECIALTY CLINIC,9.931539e+09,2.0,740 BARTLETT RD,NaN,NaN,MEMPHIS,TN,381223004,9.016858e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,TN381223004ME740XXRDXX300
2286118,1902810906,2668426438,I20050318000584,WILDER,PAULA,F,NaN,F,NP,OTHER,1998.0,NURSE PRACTITIONER,NaN,NaN,NaN,NaN,NaN,BLUFF CITY MULTISPECIALTY CLINIC,9.931539e+09,2.0,740 BARTLETT RD,NaN,NaN,MEMPHIS,TN,381223004,9.016858e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,TN381223004ME740XXRDXX300
2286119,1538325907,1355417528,I20091221000163,JOHNSON,STACI,L,NaN,F,NaN,OTHER,2008.0,NURSE PRACTITIONER,NaN,NaN,NaN,NaN,NaN,ALASKA PAIN TO WELLNESS CENTRE LLC,9.931539e+09,2.0,2751 DEBARR RD,SUITE 310,NaN,ANCHORAGE,AK,995086804,9.072780e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,AK995086804AN2751XRDXX303


In [4]:
cols = ['NPI','frst_nm',  'mid_nm', 'lst_nm', 'suff', 'gndr', 'Cred', 'pri_spec', 'cty', 'st', 'zip']

RawData = RawData[cols]
RawData = RawData.drop_duplicates().reset_index(drop = True)
RawData

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip
0,1225027097,CHARLES,G,WELCH,NaN,M,NaN,GENERAL SURGERY,CODY,WY,824143409
1,1225065105,DEAN,S,STEINBERGER,NaN,M,OD,OPTOMETRY,WESTLAKE VILLAGE,CA,913613131
2,1225064058,NANCY,L,VOIGHT,NaN,F,NaN,"PSYCHOLOGIST, CLINICAL",ORMOND BEACH,FL,321745062
3,1225022437,AMY,V,GORCZYNSKI,NaN,F,MD,INTERNAL MEDICINE,NEW HARTFORD,NY,134131108
4,1225039282,MICHAEL,W,GOLZ,NaN,M,NaN,CHIROPRACTIC,FRANKLIN LAKES,NJ,074171920
...,...,...,...,...,...,...,...,...,...,...,...
2041027,1891957023,STACY,M,ATHERTON,NaN,F,NaN,PODIATRY,DENVER,CO,802204000
2041028,1730110693,TERRYE,T,MASTIN,NaN,F,NP,NURSE PRACTITIONER,MEMPHIS,TN,381223004
2041029,1902810906,PAULA,F,WILDER,NaN,F,NP,NURSE PRACTITIONER,MEMPHIS,TN,381223004
2041030,1538325907,STACI,L,JOHNSON,NaN,F,NaN,NURSE PRACTITIONER,ANCHORAGE,AK,995086804


In [5]:
RawData['NPI'].value_counts()

1326534264    139
1346219185    119
1194984245     84
1467436998     75
1225072721     75
             ... 
1013523414      1
1013526508      1
1013571843      1
1023539350      1
1710963681      1
Name: NPI, Length: 1171890, dtype: int64

# Credential

In [6]:
RawData['Cred'].value_counts()#.iloc[:10]

MD     390457
PA      34524
NP      25419
DO      24191
CNA     18110
DC      13797
OD      12451
PT      12340
DPM      8424
CSW      7598
CP       6334
OT       1365
AU       1208
CNS      1007
CNM       891
MNT       813
DDS       437
AA        337
DDM       208
PSY       115
SCW         3
Name: Cred, dtype: int64

# Focus on MD First

In [7]:
RawData = RawData[RawData['Cred'] == 'MD'].reset_index(drop = True)
RawData

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip
0,1225022437,AMY,V,GORCZYNSKI,NaN,F,MD,INTERNAL MEDICINE,NEW HARTFORD,NY,134131108
1,1225086861,JOHN,NaN,COON,NaN,M,MD,GENERAL SURGERY,RIVERSIDE,CA,925033947
2,1225067465,NAHID,NaN,HAMOUI,NaN,F,MD,GENERAL SURGERY,IRVINE,CA,926183706
3,1225102262,CYNTHIA,S,HAYES,NaN,F,MD,FAMILY MEDICINE,COLUMBIA,MO,652034411
4,1225050552,SAMIR,I,HAMATI,NaN,M,MD,OBSTETRICS/GYNECOLOGY,NORTON SHORES,MI,494447841
...,...,...,...,...,...,...,...,...,...,...,...
390452,1003861915,MEENAKSHI,NaN,GOPAL,NaN,F,MD,ADDICTION MEDICINE,SOUTH PRINCE GEORGE,VA,238057333
390453,1003861915,MEENAKSHI,NaN,GOPAL,NaN,F,MD,ADDICTION MEDICINE,WINCHESTER,VA,226024596
390454,1770699613,SHAZIA,HABIB,FAROOQI,NaN,F,MD,FAMILY MEDICINE,DANVILLE,VA,245417352
390455,1770699613,SHAZIA,HABIB,FAROOQI,NaN,F,MD,FAMILY MEDICINE,FREDERICKSBURG,VA,224074800


# Multiple NPI

In [8]:
UniqueNPI = RawData['NPI'].value_counts()
UniqueNPI

1225072721    75
1467436998    75
1336174036    71
1265408579    70
1104840636    68
              ..
1306894555     1
1336118843     1
1538196548     1
1497860365     1
1710963681     1
Name: NPI, Length: 192113, dtype: int64

In [9]:
UniqueNPI.iloc[:10]

1225072721    75
1467436998    75
1336174036    71
1265408579    70
1104840636    68
1548218829    66
1477552933    66
1386671592    64
1982623229    62
1982678553    60
Name: NPI, dtype: int64

In [10]:
RawData[RawData['NPI'] == 1386671592]

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip
153845,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,GERMANTOWN,WI,530223109
153848,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,WHITEFISH BAY,WI,532175222
153849,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,WEST ALLIS,WI,532272455
153850,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,FRANKLIN,WI,531329083
153851,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,MILWAUKEE,WI,532331305
...,...,...,...,...,...,...,...,...,...,...,...
279784,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,CEDAR GROVE,WI,530131611
314056,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,SAINT FRANCIS,WI,532356053
314057,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,GREENFIELD,WI,532204800
314058,1386671592,JEFFREY,M,HARTWICK,NaN,M,MD,DIAGNOSTIC RADIOLOGY,OAK CREEK,WI,531544563


In [14]:
MDData = RawData.groupby('NPI').first().reset_index()
MDData

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip
0,1003002312,PATRICIA,T,HOPKINS,None,F,MD,RHEUMATOLOGY,WEYMOUTH,MA,021901843
1,1003003856,CYNTHIA,None,SANTIAGO,None,F,MD,FAMILY MEDICINE,PORT JEFFERSON STATION,NY,117763336
2,1003004540,JENNINE,M,CABANELLAS,None,F,MD,OPHTHALMOLOGY,BOYNTON BEACH,FL,334722900
3,1003004839,BEVERLY,None,DARNELL,None,F,MD,INTERNAL MEDICINE,GAITHERSBURG,MD,208793301
4,1003010349,WISSAM,None,SHAYA,None,M,MD,FAMILY MEDICINE,JACKSONVILLE,FL,322576013
...,...,...,...,...,...,...,...,...,...,...,...
192108,1992981823,JACK,None,SKARBINSKI,None,M,MD,INFECTIOUS DISEASE,OAKLAND,CA,946115312
192109,1992985360,KRZYSZTOF,A,BUJARSKI,None,M,MD,NEUROLOGY,LEBANON,NH,037561000
192110,1992987358,MARIA,L,FONDAL,None,F,MD,RHEUMATOLOGY,STOCKBRIDGE,GA,302819076
192111,1992995765,DEBRA,A,APPEL,None,F,MD,PSYCHIATRY,MOUNT KISCO,NY,105493417


In [17]:
MDData = MDData.fillna('NaN')
MDData.to_pickle('Output/MDData.p')
MDData

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip
0,1003002312,PATRICIA,T,HOPKINS,NaN,F,MD,RHEUMATOLOGY,WEYMOUTH,MA,021901843
1,1003003856,CYNTHIA,NaN,SANTIAGO,NaN,F,MD,FAMILY MEDICINE,PORT JEFFERSON STATION,NY,117763336
2,1003004540,JENNINE,M,CABANELLAS,NaN,F,MD,OPHTHALMOLOGY,BOYNTON BEACH,FL,334722900
3,1003004839,BEVERLY,NaN,DARNELL,NaN,F,MD,INTERNAL MEDICINE,GAITHERSBURG,MD,208793301
4,1003010349,WISSAM,NaN,SHAYA,NaN,M,MD,FAMILY MEDICINE,JACKSONVILLE,FL,322576013
...,...,...,...,...,...,...,...,...,...,...,...
192108,1992981823,JACK,NaN,SKARBINSKI,NaN,M,MD,INFECTIOUS DISEASE,OAKLAND,CA,946115312
192109,1992985360,KRZYSZTOF,A,BUJARSKI,NaN,M,MD,NEUROLOGY,LEBANON,NH,037561000
192110,1992987358,MARIA,L,FONDAL,NaN,F,MD,RHEUMATOLOGY,STOCKBRIDGE,GA,302819076
192111,1992995765,DEBRA,A,APPEL,NaN,F,MD,PSYCHIATRY,MOUNT KISCO,NY,105493417


# Generate Keywords

In [18]:
def get_doctor_info(DoctorDF):
    doctor_infolist = []
    for _, row in DoctorDF.iterrows():
        # most results: fistname is before lastname
        if row['mid_nm'] == "NaN":
            doctor_info = "{} {} {} {} {} {}".format(row['frst_nm'], row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  row['zip'])
        else:
            doctor_info = "{} {} {} {} {} {} {}".format(row['frst_nm'], row['mid_nm'],  row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  row['zip'])
            
            
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist

In [19]:
MDData['keyword'] = get_doctor_info(MDData)

In [23]:
MDData[['NPI', 'keyword']]

,NPI,keyword
0,1003002312,"[1003002312, PATRICIA T HOPKINS RHEUMATOLOGY W..."
1,1003003856,"[1003003856, CYNTHIA SANTIAGO FAMILY MEDICINE ..."
2,1003004540,"[1003004540, JENNINE M CABANELLAS OPHTHALMOLOG..."
3,1003004839,"[1003004839, BEVERLY DARNELL INTERNAL MEDICINE..."
4,1003010349,"[1003010349, WISSAM SHAYA FAMILY MEDICINE JACK..."
...,...,...
192108,1992981823,"[1992981823, JACK SKARBINSKI INFECTIOUS DISEAS..."
192109,1992985360,"[1992985360, KRZYSZTOF A BUJARSKI NEUROLOGY LE..."
192110,1992987358,"[1992987358, MARIA L FONDAL RHEUMATOLOGY STOCK..."
192111,1992995765,"[1992995765, DEBRA A APPEL PSYCHIATRY MOUNT KI..."


In [22]:
MDData[['NPI', 'keyword']].to_pickle('Output/MDDataKeywords.p')

# Split Half and Half

In [27]:
old_md_doc_dir = 'Data/MD_DocListDF_All.p'


Old_MD_Doc = pd.read_pickle(old_md_doc_dir)
existed_npi = Old_MD_Doc.iloc[:100000]['NPI'].unique()
len(existed_npi)

60483

In [30]:
Second_MDData = MDData[-MDData['NPI'].isin(existed_npi)].reset_index(drop = True)
Second_MDData

,NPI,frst_nm,mid_nm,lst_nm,suff,gndr,Cred,pri_spec,cty,st,zip,keyword
0,1003004540,JENNINE,M,CABANELLAS,NaN,F,MD,OPHTHALMOLOGY,BOYNTON BEACH,FL,334722900,"[1003004540, JENNINE M CABANELLAS OPHTHALMOLOG..."
1,1003004839,BEVERLY,NaN,DARNELL,NaN,F,MD,INTERNAL MEDICINE,GAITHERSBURG,MD,208793301,"[1003004839, BEVERLY DARNELL INTERNAL MEDICINE..."
2,1003010703,MACE,BENJAMIN,BRINDLEY,NaN,M,MD,OTOLARYNGOLOGY,WACO,TX,767105591,"[1003010703, MACE BENJAMIN BRINDLEY OTOLARYNGO..."
3,1003013533,GEORGE,A,CIOFFI,NaN,M,MD,OPHTHALMOLOGY,NEW YORK,NY,100323797,"[1003013533, GEORGE A CIOFFI OPHTHALMOLOGY NEW..."
4,1003015207,ROBERT,E,SWANEY,III,M,MD,HEMATOLOGY/ONCOLOGY,GREELEY,CO,806315154,"[1003015207, ROBERT E SWANEY HEMATOLOGY/ONCOLO..."
...,...,...,...,...,...,...,...,...,...,...,...,...
131625,1992974174,RAAID,NaN,MUSEITIF,NaN,M,MD,INTERVENTIONAL CARDIOLOGY,KENOSHA,WI,531422931,"[1992974174, RAAID MUSEITIF INTERVENTIONAL CAR..."
131626,1992981823,JACK,NaN,SKARBINSKI,NaN,M,MD,INFECTIOUS DISEASE,OAKLAND,CA,946115312,"[1992981823, JACK SKARBINSKI INFECTIOUS DISEAS..."
131627,1992985360,KRZYSZTOF,A,BUJARSKI,NaN,M,MD,NEUROLOGY,LEBANON,NH,037561000,"[1992985360, KRZYSZTOF A BUJARSKI NEUROLOGY LE..."
131628,1992987358,MARIA,L,FONDAL,NaN,F,MD,RHEUMATOLOGY,STOCKBRIDGE,GA,302819076,"[1992987358, MARIA L FONDAL RHEUMATOLOGY STOCK..."


In [31]:
Second_MDData[['NPI', 'keyword']].to_pickle('Output/Second_MDDataKeywords.p')